In [169]:
import numpy as np 
from numpy import diag, inf
from numpy import copy, dot
from numpy.linalg import norm
from numpy.linalg import eig
from datetime import datetime
import matplotlib.pyplot as plt

#generate a 500*500 npsd matrix 
def g_matrix(n):
    npsdm = np.full((n,n),0.9)
    for i in range(n):
        npsdm[i,i]=float(1.0)
    npsdm[0,1]=0.7357
    npsdm[1,0]=0.7357
    return npsdm
#check matrix
#check eigenvalue if it is negative 

# w is eigenvalue, v is eigenvector 

#print(w)

In [142]:
def chol_psd(m):
    root = np.full(m.shape, 0.0)
    n = root.shape[1]
    for j in range(n):
        diag_val = m[j,j] - root[j,:j] @ root[j,:j].T
        if -1e-6 <= diag_val <= 0:
            diag_val = 0.0
        elif diag_val < -1e-6:
            raise ValueError("Matrix is none-psd")
        root[j,j] = np.sqrt(diag_val)
        if root[j,j] == 0:
            continue
        for i in range(j+1, n):
            root[i, j] = (m[i,j] - root[i,:j] @ root[j,:j].T) / root[j, j] 

    return np.matrix(root)
    

In [143]:
def near_psd(m):
    w, v = np.linalg.eigh(m)
    w[w<0] = 0.0
    s_square = np.square(v)
    T = 1 / (s_square @ w)
    T = np.diagflat(np.sqrt(T))
    L = np.diag(np.sqrt(w))
    B = T @ v @ L 
    out = B @ B.T # B * B'
    return out

#check function near_psd 

In [162]:
def higham(A, tol=[], flag=0, max_iterations=100, n_pos_eig=0,
             weights=None, verbose=False,
             except_on_too_many_iterations=True):

    # If input is an ExceededMaxIterationsError object this
    # is a restart computation
    if (isinstance(A, ValueError)):
        ds = copy(A.ds)
        A = copy(A.matrix)
    else:
        ds = np.zeros(np.shape(A))

    eps = np.spacing(1)
    if not np.all((np.transpose(A) == A)):
        raise ValueError('Input Matrix is not symmetric')
    if not tol:
        tol = eps * np.shape(A)[0] * np.array([1, 1])
    if weights is None:
        weights = np.ones(np.shape(A)[0])
    X = copy(A)
    Y = copy(A)
    rel_diffY = inf
    rel_diffX = inf
    rel_diffXY = inf

    Whalf = np.sqrt(np.outer(weights, weights))

    iteration = 0
    while max(rel_diffX, rel_diffY, rel_diffXY) > tol[0]:
        iteration += 1
        if iteration > max_iterations:
            if except_on_too_many_iterations:
                if max_iterations == 1:
                    message = "No solution found in "\
                              + str(max_iterations) + " iteration"
                else:
                    message = "No solution found in "\
                              + str(max_iterations) + " iterations"
                raise ExceededMaxIterationsError(message, X, iteration, ds)
            else:
                # exceptOnTooManyIterations is false so just silently
                # return the result even though it has not converged
                return X

        Xold = copy(X)
        R = X - ds
        R_wtd = Whalf*R
        if flag == 0:
            X = proj_spd(R_wtd)
        elif flag == 1:
            raise NotImplementedError("Setting 'flag' to 1 is currently\
                                 not implemented.")
        X = X / Whalf
        ds = X - R
        Yold = copy(Y)
        Y = copy(X)
        np.fill_diagonal(Y, 1)
        normY = norm(Y, 'fro')
        rel_diffX = norm(X - Xold, 'fro') / norm(X, 'fro')
        rel_diffY = norm(Y - Yold, 'fro') / normY
        rel_diffXY = norm(Y - X, 'fro') / normY

        X = copy(Y)

    return X

def proj_spd(A):
    # NOTE: the input matrix is assumed to be symmetric
    d, v = np.linalg.eigh(A)
    A = (v * np.maximum(d, 0)).dot(v.T)
    A = (A + A.T) / 2
    return(A)

#hns = higham(ns)
#print(hns)

In [145]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x)>0)

def check_psd(matrix, tolerance=1e-6):
    eig_values, eig_vectors = np.linalg.eigh(matrix)
    return all(eig_values > -tolerance)

try_1 = g_matrix(5)
print(check_psd(try_1))
print(check_psd(hns))
print(check_psd(nps))

False
True
True


In [146]:
#frobenius norm 

450.10465662398644
450.0494394140792


In [212]:
def autorun(n): 
# time for near_psd 
    start_time = datetime.now()
    nsa=g_matrix(n)
    w,v=eig(nsa)
    nnsa=near_psd(nsa)
    near_psf_fro = (np.linalg.norm((nsa-nnsa),'fro'))
    end_time = datetime.now()
    NDuration = end_time - start_time
    
    start_time = datetime.now()
    nsb=g_matrix(n)
    w,v=eig(nsb)
    hnps=higham(nsb)
    higham_fro= (np.linalg.norm((nsb-hnps)),'fro')
    end_time = datetime.now()
    hDuration= end_time - start_time
    return near_psf_fro,NDuration,higham_fro,hDuration

In [216]:
array = [500,600,700,800,900,1000]
for i in array: 
    a,b,c,d = autorun(i)
    print("Size is ", i, "\n")
    print("Duration of Chloesky is ", b, "Duration of Higham is ", d,"\n")
    print("F Norm of Chloesky is ", a, "F Norm  of Higham is ", c,"\n")

Size is  500 

Duration of Chloesky is  0:00:00.209020 Duration of Higham is  0:00:01.275974 

F Norm of Chloesky is  0.6275226557659508 F Norm  of Higham is  (0.08964799632524456, 'fro') 

Size is  600 

Duration of Chloesky is  0:00:00.286990 Duration of Higham is  0:00:01.875891 

F Norm of Chloesky is  0.6882026666308428 F Norm  of Higham is  (0.08986132327064164, 'fro') 

Size is  700 

Duration of Chloesky is  0:00:00.342976 Duration of Higham is  0:00:02.952299 

F Norm of Chloesky is  0.7439495037336205 F Norm  of Higham is  (0.09001394198185893, 'fro') 

Size is  800 

Duration of Chloesky is  0:00:00.486832 Duration of Higham is  0:00:03.536592 

F Norm of Chloesky is  0.7958006315839199 F Norm  of Higham is  (0.09012853893086391, 'fro') 

Size is  900 

Duration of Chloesky is  0:00:00.584460 Duration of Higham is  0:00:04.631598 

F Norm of Chloesky is  0.8444739843546087 F Norm  of Higham is  (0.09021774889611904, 'fro') 

Size is  1000 

Duration of Chloesky is  0:00:00.7